In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.decomposition import PCA

In [2]:
df=pd.read_csv("final.csv",index_col=0)

In [3]:
df.head()

,Genre,Title,Artist,Explicit,Length,Scale,Key,BPM,Danceability,Positiveness,Liveness,Energy,speechness,KeyScaled,Lyrics,Not_Formatted_Lyrics,Lem_Lyrics,Lyrics_Length,Longest_Line,Length_Words_Ratio
0,rock,into the storm,gojira,False,5.02,D,Major,106,25,11,13,97,7,D,go say something about awakening the rebellion...,Go\r\n\r\nSay something about awakening\r\nThe...,awaken rebellion wake change course law follow...,173,47,34.462151
1,rock,the sound of silence,disturbed,False,4.08,F#,Minor,86,32,18,10,28,3,F#m,hello darkness my old friend i have come to ta...,"Hello darkness, my old friend\r\nI've come to ...",hello darkness old friend I talk vision softly...,218,97,53.431373
2,rock,amazonia,gojira,False,5.00,A,Major,182,38,51,24,95,4,A,incite a riot put yourself in a trance you rot...,"Incite a riot, put yourself in a trance\r\nYou...",incite riot put trance rotate frame world rely...,154,48,30.800000
3,rock,back in black,ac/dc,False,4.15,A,Major,188,31,76,8,70,5,A,back in black i hit the sack i have been too l...,"Back in black, I hit the sack\r\nI've been too...",back black I hit sack I long I glad back yes I...,307,55,73.975904
4,rock,another world,gojira,False,4.24,G,Minor,170,35,24,12,88,4,Gm,we mock and slaughter all the purest kinds bli...,We mock and slaughter all the purest kinds\r\n...,mock slaughter pure kind blind noise maze flas...,95,53,22.405660


In [5]:
df.columns

Index(['Genre', 'Title', 'Artist', 'Explicit', 'Length', 'Scale', 'Key', 'BPM',
       'Danceability', 'Positiveness', 'Liveness', 'Energy', 'speechness',
       'KeyScaled', 'Lyrics', 'Not_Formatted_Lyrics', 'Lem_Lyrics',
       'Lyrics_Length', 'Longest_Line', 'Length_Words_Ratio'],
      dtype='object')

In [6]:
df.drop(['Lyrics','KeyScaled','Not_Formatted_Lyrics'],axis=1,inplace=True)

In [8]:
df.head()

,Genre,Title,Artist,Explicit,Length,Scale,Key,BPM,Danceability,Positiveness,Liveness,Energy,speechness,Lem_Lyrics,Lyrics_Length,Longest_Line,Length_Words_Ratio
0,rock,into the storm,gojira,False,5.02,D,Major,106,25,11,13,97,7,awaken rebellion wake change course law follow...,173,47,34.462151
1,rock,the sound of silence,disturbed,False,4.08,F#,Minor,86,32,18,10,28,3,hello darkness old friend I talk vision softly...,218,97,53.431373
2,rock,amazonia,gojira,False,5.00,A,Major,182,38,51,24,95,4,incite riot put trance rotate frame world rely...,154,48,30.800000
3,rock,back in black,ac/dc,False,4.15,A,Major,188,31,76,8,70,5,back black I hit sack I long I glad back yes I...,307,55,73.975904
4,rock,another world,gojira,False,4.24,G,Minor,170,35,24,12,88,4,mock slaughter pure kind blind noise maze flas...,95,53,22.405660


In [9]:
df["Major"] = df["Key"]
df["Major"] = df["Major"].apply(lambda x: 0 if x == "Minor" else 1)
df.drop(columns=['Key'],inplace=True)

In [10]:
res = pd.get_dummies(df["Scale"])
df.drop(columns=['Scale'],inplace=True)
combine = pd.concat([df, res], axis=1)
combine

,Genre,Title,Artist,Explicit,Length,BPM,Danceability,Positiveness,Liveness,Energy,...,B,C,C#,D,D#,E,F,F#,G,G#
0,rock,into the storm,gojira,False,5.02,106,25,11,13,97,...,0,0,0,1,0,0,0,0,0,0
1,rock,the sound of silence,disturbed,False,4.08,86,32,18,10,28,...,0,0,0,0,0,0,0,1,0,0
2,rock,amazonia,gojira,False,5.00,182,38,51,24,95,...,0,0,0,0,0,0,0,0,0,0
3,rock,back in black,ac/dc,False,4.15,188,31,76,8,70,...,0,0,0,0,0,0,0,0,0,0
4,rock,another world,gojira,False,4.24,170,35,24,12,88,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11394,hiphop,higher learning,young jeezy,True,3.44,77,71,90,11,65,...,0,0,0,0,0,0,0,0,0,1
11395,hiphop,its goin down,celly cel,False,5.24,75,82,78,4,50,...,0,0,0,0,0,0,0,1,0,0
11398,hiphop,let me know,cam'ron,True,4.14,92,75,69,35,79,...,0,0,0,0,0,0,0,1,0,0
11401,hiphop,beta,tyson james,False,3.54,89,73,58,28,57,...,0,0,0,0,0,0,0,0,1,0


In [24]:
combine.iloc[:10,:16]

,Genre,Title,Artist,Explicit,Length,BPM,Danceability,Positiveness,Liveness,Energy,speechness,Lem_Lyrics,Lyrics_Length,Longest_Line,Length_Words_Ratio,Major
0,rock,into the storm,gojira,False,5.02,106,25,11,13,97,7,awaken rebellion wake change course law follow...,173,47,34.462151,1
1,rock,the sound of silence,disturbed,False,4.08,86,32,18,10,28,3,hello darkness old friend I talk vision softly...,218,97,53.431373,0
2,rock,amazonia,gojira,False,5.00,182,38,51,24,95,4,incite riot put trance rotate frame world rely...,154,48,30.800000,1
3,rock,back in black,ac/dc,False,4.15,188,31,76,8,70,5,back black I hit sack I long I glad back yes I...,307,55,73.975904,1
4,rock,another world,gojira,False,4.24,170,35,24,12,88,4,mock slaughter pure kind blind noise maze flas...,95,53,22.405660,0
6,rock,don't stop believin',journey,False,4.10,119,50,51,45,75,4,smalltown girl livin ' lonely world midnight t...,190,51,46.341463,1
7,rock,eye of the tiger,survivor,False,4.05,109,82,55,9,60,3,rise back street chance distance I back foot m...,276,46,68.148148,0
8,rock,africa,toto,False,4.55,93,67,73,5,37,3,I hear drum echo tonight hear whisper quiet co...,280,67,61.538462,1
9,rock,boilermaker,royal blood,False,3.29,95,45,68,5,78,17,I look kind saviour still count bad behaviour ...,217,66,65.957447,1
10,rock,born for one thing,gojira,False,4.20,146,22,38,31,97,8,mortal spawn altar catch heart blaze primal fe...,154,46,36.666667,1


In [8]:
X=combine.drop(columns=['Genre','Title','Artist','Lem_Lyrics'])
y=combine['Genre']
X

,Explicit,Length,BPM,Danceability,Positiveness,Liveness,Energy,speechness,Lyrics_Length,Major,...,E,Em,F,F#,F#m,Fm,G,G#,G#m,Gm
0,False,5.02,106,25,11,13,97,7,173,1,...,0,0,0,0,0,0,0,0,0,0
1,False,4.08,86,32,18,10,28,3,218,0,...,0,0,0,0,1,0,0,0,0,0
2,False,5.00,182,38,51,24,95,4,154,1,...,0,0,0,0,0,0,0,0,0,0
3,False,4.15,188,31,76,8,70,5,307,1,...,0,0,0,0,0,0,0,0,0,0
4,False,4.24,170,35,24,12,88,4,95,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11398,True,4.14,92,75,69,35,79,33,765,1,...,0,0,0,1,0,0,0,0,0,0
11399,True,4.04,81,82,68,13,83,27,663,1,...,0,0,0,1,0,0,0,0,0,0
11400,True,4.19,95,68,39,13,72,8,577,1,...,0,0,0,0,0,0,1,0,0,0
11401,False,3.54,89,73,58,28,57,45,743,1,...,0,0,0,0,0,0,1,0,0,0


In [9]:

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)


In [10]:
text_mnb=SVC().fit(X_train,y_train)

In [11]:
ba=text_mnb.predict(X_test)
print(accuracy_score(y_true=y_test,y_pred=ba))

0.45500459136822774
